In [1]:
import numpy as np

from sklearn.metrics import accuracy_score
from keras.datasets import reuters
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, LSTM, Activation, GRU
from keras import optimizers


In [2]:
# parameters for data load
num_words = 30000
maxlen = 50
test_split = 0.3

In [3]:
(X_train, y_train), (X_test, y_test) = reuters.load_data(num_words = num_words, maxlen = maxlen, test_split = test_split)

2110848/2110848 [==============================] - 0s 0us/step


In [4]:
# pad the sequences with zeros
# padding parameter is set to 'post' => 0's are appended to end of sequences
X_train = pad_sequences(X_train, padding = 'post')
X_test = pad_sequences(X_test, padding = 'post')

X_train = np.array(X_train).reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = np.array(X_test).reshape((X_test.shape[0], X_test.shape[1], 1))

y_data = np.concatenate((y_train, y_test))
y_data = to_categorical(y_data)

y_actual = y_test

y_train = y_data[:1395]
y_test = y_data[1395:]

In [5]:
X_train.shape

(1395, 49, 1)

In [6]:
y_data.shape

(1994, 46)

## 1. Deep LSTM
- LSTMs can be made deep, with multiple layers, like CNNs or MLPs
- Beware that LSTMs take long to train compared to CNNs

In [7]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Activation, GRU
from keras import optimizers


In [8]:
def deep_lstm():
    model = Sequential()
    model.add(LSTM(20, input_shape = (49,1), return_sequences = True))
    model.add(LSTM(20, return_sequences = True))
    model.add(LSTM(20, return_sequences = True))
    model.add(LSTM(20, return_sequences = False))
    model.add(Dense(46))
    model.add(Activation('softmax'))

    adam = optimizers.Adam(lr = 0.001)
    model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])

    return model

In [9]:
model = deep_lstm()
model.fit(X_train, y_train, epochs = 200, batch_size = 50, verbose = 1)

Epoch 1/200
28/28 [==============================] - 8s 17ms/step - loss: 3.3088 - accuracy: 0.6573
Epoch 2/200
28/28 [==============================] - 0s 16ms/step - loss: 2.0167 - accuracy: 0.7147
Epoch 3/200
28/28 [==============================] - 0s 16ms/step - loss: 1.3961 - accuracy: 0.7147
Epoch 4/200
28/28 [==============================] - 0s 18ms/step - loss: 1.2565 - accuracy: 0.7147
Epoch 5/200
28/28 [==============================] - 0s 17ms/step - loss: 1.2118 - accuracy: 0.7147
Epoch 6/200
28/28 [==============================] - 0s 13ms/step - loss: 1.1910 - accuracy: 0.7147
Epoch 7/200
28/28 [==============================] - 0s 12ms/step - loss: 1.1797 - accuracy: 0.7147
Epoch 8/200
28/28 [==============================] - 0s 12ms/step - loss: 1.1741 - accuracy: 0.7147
Epoch 9/200
28/28 [==============================] - 0s 13ms/step - loss: 1.1693 - accuracy: 0.7147
Epoch 10/200
28/28 [==============================] - 0s 12ms/step - loss: 1.1666 - accuracy: 0.7147

In [10]:
y_pred = model.predict(X_test)
y_test_ = np.argmax(y_pred, axis = 1)

19/19 [==============================] - 1s 6ms/step


In [11]:
print(accuracy_score(y_actual, y_test_))

0.8480801335559266


## 2. Bidirectional RNN
- Bidirectional RNNs consider not only one-way influence of sequence, but also the other way
- Actually, they can be thought as building two separate RNNs, and merging them\
<br>
<img src="http://d3kbpzbmcynnmx.cloudfront.net/wp-content/uploads/2015/09/bidirectional-rnn.png" style="width: 400px"/>
</br>

In [12]:
from keras.layers import Bidirectional

In [13]:
def bidirectional_lstm():
    model = Sequential()
    model.add(Bidirectional(LSTM(20, return_sequences = False), input_shape = (49,1)))
    model.add(Dense(46))
    model.add(Activation('softmax'))

    adam = optimizers.Adam(lr = 0.001)
    model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])

    return model

In [14]:
%%time
model = bidirectional_lstm()
model.fit(X_train, y_train, epochs = 2, batch_size = 5, verbose = 1)

Epoch 1/2
279/279 [==============================] - 5s 8ms/step - loss: 1.6265 - accuracy: 0.6416
Epoch 2/2
279/279 [==============================] - 2s 7ms/step - loss: 0.9837 - accuracy: 0.7412
CPU times: user 8.4 s, sys: 345 ms, total: 8.74 s
Wall time: 8.16 s


Epoch = passing all the data once to my network
batch = group of input sent at once
Iteration = number of time the batch of data to pass through the network to complete one epoch

sample size =  1395
Epoch = 2
batch = 5
Iteration = 1395/5 = 279

In [15]:
y_pred = model.predict(X_test)
y_test_ = np.argmax(y_pred, axis = 1)

19/19 [==============================] - 1s 3ms/step


In [16]:
print(accuracy_score(y_actual, y_test_))

0.8063439065108514


## 3. Deep Bidirectional RNN
- Bidirectional RNNs can be stacked

<img src="http://www.wildml.com/wp-content/uploads/2015/09/Screen-Shot-2015-09-16-at-2.21.51-PM-272x300.png" style="width: 300px"/>

In [17]:
def deep_bidirectional_lstm():
    model = Sequential()
    model.add(Bidirectional(LSTM(10, return_sequences = True), input_shape = (49,1)))
    model.add(Bidirectional(LSTM(10, return_sequences = True)))
    model.add(Bidirectional(LSTM(10, return_sequences = True)))
    model.add(Bidirectional(LSTM(10, return_sequences = False)))
    model.add(Dense(46))
    model.add(Activation('softmax'))

    adam = optimizers.Adam(lr = 0.001)
    model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])

    return model

In [18]:
model = deep_bidirectional_lstm()
model.fit(X_train, y_train, epochs = 200, batch_size = 50, verbose = 1)

Epoch 1/200
28/28 [==============================] - 13s 23ms/step - loss: 3.5546 - accuracy: 0.5297
Epoch 2/200
28/28 [==============================] - 1s 26ms/step - loss: 2.6887 - accuracy: 0.7147
Epoch 3/200
28/28 [==============================] - 1s 30ms/step - loss: 1.7584 - accuracy: 0.7147
Epoch 4/200
28/28 [==============================] - 1s 32ms/step - loss: 1.3013 - accuracy: 0.7147
Epoch 5/200
28/28 [==============================] - 1s 22ms/step - loss: 1.2096 - accuracy: 0.7147
Epoch 6/200
28/28 [==============================] - 1s 22ms/step - loss: 1.1869 - accuracy: 0.7147
Epoch 7/200
28/28 [==============================] - 1s 22ms/step - loss: 1.1768 - accuracy: 0.7147
Epoch 8/200
28/28 [==============================] - 1s 23ms/step - loss: 1.1690 - accuracy: 0.7147
Epoch 9/200
28/28 [==============================] - 1s 22ms/step - loss: 1.1573 - accuracy: 0.7147
Epoch 10/200
28/28 [==============================] - 1s 22ms/step - loss: 1.0690 - accuracy: 0.714

In [19]:
y_pred = model.predict(X_test)
y_test_ = np.argmax(y_pred, axis = 1)

19/19 [==============================] - 2s 10ms/step


In [20]:
print(accuracy_score(y_actual, y_test_))

0.8063439065108514


In [ ]:
1. SimpleRNN
2. Stacked RNN
3. LSTM
4. Stacked LSTM
5. Bidirectional LSTM
6. Stacked Bidirectional LSTM

## Bidirectional GRU


In [21]:
from keras.layers import Bidirectional

In [22]:
def bidirectional_gru():
    model = Sequential()
    model.add(Bidirectional(GRU(20, return_sequences = False), input_shape = (49,1)))
    model.add(Dense(46))
    model.add(Activation('softmax'))

    adam = optimizers.Adam()
    model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])

    return model

In [23]:
%%time
model = bidirectional_gru()
model.fit(X_train, y_train, epochs = 2, batch_size = 5, verbose = 1)

Epoch 1/2
279/279 [==============================] - 5s 9ms/step - loss: 1.6129 - accuracy: 0.6452
Epoch 2/2
279/279 [==============================] - 2s 7ms/step - loss: 1.0838 - accuracy: 0.7133
CPU times: user 7.85 s, sys: 371 ms, total: 8.22 s
Wall time: 7.75 s


In [24]:
y_pred = model.predict(X_test)
y_test_ = np.argmax(y_pred, axis = 1)

19/19 [==============================] - 1s 5ms/step


In [25]:
print(accuracy_score(y_actual, y_test_))

0.7646076794657763
